In [1]:
# # Install:
# # GFootball environment (https://github.com/google-research/football/),
# # SEED RL for training an agent (https://github.com/google-research/seed_rl/),
# # Tensorflow 2.2, which is needed by SEED RL.

# !apt-get update
# !apt-get install -y libsdl2-gfx-dev libsdl2-ttf-dev
# !pip3 install tensorflow==2.2
# !pip3 install tensorflow_probability==0.9.0

# # Update kaggle-environments to the newest version.
# !pip3 install kaggle-environments -U

# # Make sure that the Branch in git clone and in wget call matches !!
# !git clone -b v2.6 https://github.com/google-research/football.git
# !mkdir -p football/third_party/gfootball_engine/lib

# !wget https://storage.googleapis.com/gfootball/prebuilt_gameplayfootball_v2.6.so -O football/third_party/gfootball_engine/lib/prebuilt_gameplayfootball.so
# !cd football && GFOOTBALL_USE_PREBUILT_SO=1 pip3 install .

# !git clone https://github.com/higepon/seed_rl.git
# !cd seed_rl && git checkout kaggle_gfootball2

In [2]:
# Kaggle environment
!git clone https://github.com/Kaggle/kaggle-environments.git
!cd kaggle-environments && pip3 install -q .

# GFootball requirement
!apt-get update -y
!apt-get install -y libsdl2-gfx-dev libsdl2-ttf-dev

# GFootball environment
# Make sure that the Branch in git clone and in wget call matches !!
!git clone -b v2.5 https://github.com/google-research/football.git
!mkdir -p football/third_party/gfootball_engine/lib
!wget https://storage.googleapis.com/gfootball/prebuilt_gameplayfootball_v2.5.so -O football/third_party/gfootball_engine/lib/prebuilt_gameplayfootball.so
# Custom files
!wget https://gist.githubusercontent.com/RaffaeleMorganti/04192739d0a5a518ac253889eb83c6f1/raw/c09f3d602ea89e66daeda96574d966949a2896ce/11_vs_11_deterministic.py -O football/gfootball/scenarios/11_vs_11_deterministic.py 
!wget https://gist.githubusercontent.com/RaffaeleMorganti/04192739d0a5a518ac253889eb83c6f1/raw/c09f3d602ea89e66daeda96574d966949a2896ce/football_action_set.py -O football/gfootball/env/football_action_set.py
!cd football && GFOOTBALL_USE_PREBUILT_SO=1 pip3 install -q .

Cloning into 'kaggle-environments'...
remote: Enumerating objects: 1298, done.
remote: Counting objects: 100% (1298/1298), done.
remote: Compressing objects: 100% (932/932), done.
remote: Total 3199 (delta 458), reused 1122 (delta 328), pack-reused 1901
Receiving objects: 100% (3199/3199), 9.26 MiB | 11.97 MiB/s, done.
Resolving deltas: 100% (1831/1831), done.
     |████████████████████████████████| 61kB 3.2MB/s 
ERROR: nbclient 0.5.1 has requirement jupyter-client>=6.1.5, but you'll have jupyter-client 5.3.5 which is incompatible.
Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Get:6 

In [3]:
# !cat football/gfootball/scenarios/11_vs_11_hard_stochastic.py

In [4]:
# %%bash
# cd football && GFOOTBALL_USE_PREBUILT_SO=1 pip3 install -qq . && cd ..
# rm -rf football

In [5]:
from pathlib import Path

def mount_drive_if_necessary(drive_root):
    from google.colab import drive
    if not drive_root.exists():
        drive.mount(str(drive_root))

def on_colab():
    return Path("/content").exists()        

def on_colab():
    return Path("/content").exists()        

def root_dir():
    if on_colab():
        drive_root_dir = Path('/content/drive')
        mount_drive_if_necessary(drive_root_dir)
        return drive_root_dir / "My Drive" / "Gfootball"
    else:
        return Path("/sUsers/higepon/Google Drive/kaggle/google-football")

# This is the only global variable
project_dir = root_dir()
print(f"{project_dir}")
!nvidia-smi | grep 'Tesla' | cut -d ' ' -f 7

Mounted at /content/drive
/content/drive/My Drive/Gfootball


In [6]:
output_dir = project_dir / 'experiments/SEED_hard_plus_bot_1625M/1'
!mkdir -p '$output_dir'
!ln -sf "$output_dir" training_state
!mkdir training_state

mkdir: cannot create directory ‘training_state’: File exists


In [7]:
!mkdir -p /kaggle/working
!mkdir -p  /kaggle_simulations/agent
!cp -r training_state/*  /kaggle_simulations/agent/
!curl -Ls https://github.com/higepon/seed_rl/releases/download/0.0.1/model.sav.zip -o model.sav.zip
!unzip model.sav.zip
!mv model.sav training_state

Archive:  model.sav.zip
  inflating: model.sav               
  inflating: __MACOSX/._model.sav    


In [8]:
%%writefile submission.py
from kaggle_environments.envs.football.helpers import *
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
import pickle
import math

# load the model from disk
filename = 'training_state/model.sav'
loaded_model = pickle.load(open(filename, 'rb'))
#result = loaded_model.score(X_test, y_test)
#print(result)

directions = [[Action.TopLeft, Action.Top, Action.TopRight],
[Action.Left, Action.Idle, Action.Right],
[Action.BottomLeft, Action.Bottom, Action.BottomRight]]

#track raw data to troubleshoot...
track_raw_data=[]

perfectRange = [[0.7, 0.95], [-0.12, 0.12]]

def inside(pos, area):
    return area[0][0] <= pos[0] <= area[0][1] and area[1][0] <= pos[1] <= area[1][1]

def get_distance(pos1,pos2):
    return ((pos1[0]-pos2[0])**2+(pos1[1]-pos2[1])**2)**0.5

def get_heading(pos1,pos2):
    raw_head=math.atan2(pos1[0]-pos2[0],pos1[1]-pos2[1])/math.pi*180

    if raw_head<0:
        head=360+raw_head
    else:
        head=raw_head
    return head

def get_action(action_num):
    if action_num==0:
        return Action.Idle
    if action_num==1:
        return Action.Left
    if action_num==2:
        return Action.TopLeft
    if action_num==3:
        return Action.Top
    if action_num==4:
        return Action.TopRight
    if action_num==5:
        return Action.Right
    if action_num==6:
        return Action.BottomRight
    if action_num==7:
        return Action.Bottom
    if action_num==8:
        return Action.BottomLeft
    if action_num==9:
        return Action.LongPass
    if action_num==10:
        return Action.HighPass
    if action_num==11:
        return Action.ShortPass
    if action_num==12:
        return Action.Shot
    if action_num==13:
        return Action.Sprint
    if action_num==14:
        return Action.ReleaseDirection
    if action_num==15:
        return Action.ReleaseSprint
    if action_num==16:
        #return Action.Sliding
        return Action.Idle
    if action_num==17:
        return Action.Dribble
    if action_num==18:
        #return Action.ReleaseDribble
        return Action.Idle
    return Action.Right

@human_readable_agent
def agent(obs):
    controlled_player_pos = obs['left_team'][obs['active']]
    x = controlled_player_pos[0]
    y = controlled_player_pos[1]
    pactive=obs['active']
    
    if obs["game_mode"] == GameMode.Penalty:
        return Action.Shot
    if obs["game_mode"] == GameMode.Corner:
        if controlled_player_pos[0] > 0:
            return Action.Shot
    if obs["game_mode"] == GameMode.FreeKick:
        return Action.Shot
    
    # Make sure player is running.
    if  0 < controlled_player_pos[0] < 0.6 and Action.Sprint not in obs['sticky_actions']:
        return Action.Sprint
    elif 0.6 < controlled_player_pos[0] and Action.Sprint in obs['sticky_actions']:
        return Action.ReleaseSprint
    
    #if we have the ball
    if obs['ball_owned_player'] == obs['active'] and obs['ball_owned_team'] == 0:
        dat=[]
        to_append=[]
        #return Action.Right
        #get controller player pos
        controlled_player_pos = obs['left_team'][obs['active']]
        
        if inside(controlled_player_pos, perfectRange) and controlled_player_pos[0] < obs['ball'][0]:
            return Action.Shot
        
        goalx=0.0
        goaly=0.0

        sidelinex=0.0
        sideliney=0.42
        
        goal_dist=get_distance((x,y),(goalx,goaly))
        sideline_dist=get_distance((x,y),(sidelinex,sideliney))
        to_append.append(goal_dist)
        to_append.append(sideline_dist)
        
        for i in range(len(obs['left_team'])):
            dist=get_distance((x,y),(obs['left_team'][i][0],obs['left_team'][i][1]))
            head=get_heading((x,y),(obs['left_team'][i][0],obs['left_team'][i][1]))
            to_append.append(dist)
            to_append.append(head)
            
        for i in range(len(obs['right_team'])):
            dist=get_distance((x,y),(obs['right_team'][i][0],obs['right_team'][i][1]))
            head=get_heading((x,y),(obs['right_team'][i][0],obs['right_team'][i][1]))
            to_append.append(dist)
            to_append.append(head)
        
        dat.append(to_append)
        
        predicted=loaded_model.predict(dat)
        
        do=get_action(predicted)
        
        if do == None:
            return Action.Right
        else:
            return do
    
    # if we don't have ball run to ball
    else:
        dirsign = lambda x: 1 if abs(x) < 0.01 else (0 if x < 0 else 2)
        #where ball is going
        ball_targetx=obs['ball'][0]+(obs['ball_direction'][0]*5)
        ball_targety=obs['ball'][1]+(obs['ball_direction'][1]*5)
        
        e_dist=get_distance(obs['left_team'][obs['active']],obs['ball'])
        
        if e_dist >.01:
            # Run where ball will be
            xdir = dirsign(ball_targetx - controlled_player_pos[0])
            ydir = dirsign(ball_targety - controlled_player_pos[1])
            return directions[ydir][xdir]
        else:
            # Run towards the ball.
            xdir = dirsign(obs['ball'][0] - controlled_player_pos[0])
            ydir = dirsign(obs['ball'][1] - controlled_player_pos[1])
            return directions[ydir][xdir]
        


Writing submission.py


In [9]:
%%writefile /content/training_state/main.py
import collections
import gym
import numpy as np
import os
import sys
import tensorflow as tf

from gfootball.env import observation_preprocessing
from gfootball.env import wrappers

EnvOutput = collections.namedtuple(
    'EnvOutput', 'reward done observation abandoned episode_step')

def prepare_agent_input(observation, prev_action, state):
    # SEED RL agent accepts input in a form of EnvOutput. When not training
    # only observation is used for generating action, so we use a dummy values
    # for the rest.
    env_output = EnvOutput(reward=tf.zeros(shape=[], dtype=tf.float32),
        done=tf.zeros(shape=[], dtype=tf.bool),
        observation=observation, abandoned=False,
        episode_step=tf.zeros(shape=[], dtype=tf.int32))
    # add batch dimension
    prev_action, env_output = tf.nest.map_structure(
        lambda t: tf.expand_dims(t, 0), (prev_action, env_output))

    return (prev_action, env_output, state)

# Previously executed action
previous_action = tf.constant(0, dtype=tf.int64)
# Queue of recent observations (SEED agent we trained uses frame stacking).
observations = collections.deque([], maxlen=4)
# Current state of the agent (used by recurrent agents).
state = ()

# Load previously trained Tensorflow model.
policy = tf.compat.v2.saved_model.load('/kaggle_simulations/agent/saved_model')

def agent(obs):
    global step
    global previous_action
    global observations
    global state
    global policy
    with open("./hoge.log", "w+") as g:
      try:
        # Get observations for the first (and only one) player we control.
        obs = obs['players_raw'][0]
        # Agent we trained uses Super Mini Map (SMM) representation.
        # See https://github.com/google-research/seed_rl/blob/master/football/env.py for details.
        obs = observation_preprocessing.generate_smm([obs])[0]
        g.write(f"obs={obs.shape}")
        if not observations:
            observations.extend([obs] * 4)
        else:
            observations.append(obs)
        
        obs = np.concatenate(list(observations), axis=-1)
        g.write(f"obs2={obs.shape}")
        # SEED packs observations to reduce transfer times.
        # See PackedBitsObservation in https://github.com/google-research/seed_rl/blob/master/football/observation.py
        obs = np.packbits(obs, axis=-1)
        if obs.shape[-1] % 2 == 1:
            obs = np.pad(obs, [(0, 0)] * (obs.ndim - 1) + [(0, 1)], 'constant')
        obs = obs.view(np.uint16)
        g.write(f"obs3={obs.shape}")
        # Execute our agent to obtain action to take.
        agent_output, state = policy.get_action(*prepare_agent_input(obs, previous_action, state))
        previous_action = agent_output.action[0]
        g.write(f"action={int(previous_action)}")
        return [int(previous_action)]
      except Exception as e:
        g.write(f'e={e}')

Overwriting /content/training_state/main.py


In [ ]:
from kaggle_environments import make
scores = []
for _ in range(10):
  env = make("football", debug=True,
            configuration={"save_video": True, 
                            "scenario_name": "11_vs_11_kaggle", 
                            "running_in_notebook": True})
          
  # output = env.run(["training_state/main.py", "run_right"])
  # output = env.run(["run_right", "training_state/main.py"])
  output = env.run(["/content/training_state/main.py", "submission.py"])
  scores.append(output[-1][0]["observation"]["players_raw"][0]["score"])
  # print("Scores  {0} : {1}".format(*scores))
  #print("Rewards {0} : {1}".format(output[-1][0]["reward"], output[-1][1]["reward"]))

our_total = 0
their_total = 0
num_win = 0
num_lose = 0
num_tie = 0
for our, their in scores:
  result_str = ""
  if our > their:
    num_win += 1
    result_str = "WIN"
  elif our < their:
    num_lose += 1
    result_str = "LOSE"
  else:
    num_tie += 1
    result_str = "TIE"
  print(f"Score: {our}:{their} {result_str} ")
  our_total += our
  their_total += their
print("---------------")
print(f"Totals {our_total}:{their_total} ({num_win}勝{num_lose}敗{num_tie}引き分け)")

Staring a new environment 0212eb6e-5ed7-4cda-a904-27a2d2336ef1: with scenario: 11_vs_11_kaggle
Resetting environment 0212eb6e-5ed7-4cda-a904-27a2d2336ef1: with scenario: 11_vs_11_kaggle


In [ ]:
from kaggle_environments import make
env = make("football", debug=True,
           configuration={"save_video": True, 
                          "scenario_name": "11_vs_11_kaggle", 
                          "running_in_notebook": True})
         
# output = env.run(["training_state/main.py", "run_right"])
# output = env.run(["run_right", "training_state/main.py"])
output = env.run(["/content/training_state/main.py", "submission.py"])
scores = output[-1][0]["observation"]["players_raw"][0]["score"]
print("Scores  {0} : {1}".format(*scores))
print("Rewards {0} : {1}".format(output[-1][0]["reward"], output[-1][1]["reward"]))
env.render(mode="human", width=800, height=600)